# Lancaster Housing Association Lettings

The code below downloads the data found in the historic lettings section of the Ideal Choice Homes [website](https://www.idealchoicehomes.co.uk/Data/ASPPages/1/56.aspx) which is a published list of properties that have been let in a given week.  This notebook goes through the steps of automatically downloading and collating the data.  If ran at another time the data downloaded may be different, I don't know if the published data is a rolling window or not.

It should be noted that the properties that are let may not be indicative of all the properties that were available to rent, as they may not have been let and so recorded.  If this is a situation that's possible the data will not give a complete picture of the number and type of lettings that have been made available historically, and at what times of the year.

## Library Imports

Below we import pre-existing code libraries, called modules.  Each module can provide pre-made classes and/or functions that make it easier to carry out the task by re-using existing code.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datefinder

## Web Data Scraping

In this section, code will connect to the website and after a process of link-harvesting, will download all the letting information.  There are five stages to this:

1. Connect to the first page of links to lettings, verify that this connection was successful.
2. Using the page above, identify all the paginated links, e.g. page 2, page 3, page 4 of the links to lettings.
3. Iterate through each of the pages identified in stage 2, and add each link found with the letting cycle prefix to a list.
4. For each of the URI's in the list produced by stage 3, go to each page and look for housing data, harvesting and appending to a data frame if found.
5. Write the harvested data to a CSV formatted file.

## Web Data Scraping: stage 1

In the code block below the webpage is accessed, and a print command outputs a sentence that is finished with the outcome of the request.  This check of the request's outcome is performed here for example, but at all later points it is assumed that a connection is OK.

The variable `root_address` is created, this is the base URI of the website that all the links we will programatically access are in relation to; it will be used as a prefix.

In [2]:
# Use the base links_page to point to the first of the lettings cycle list pages
links_page = requests.get("https://www.idealchoicehomes.co.uk/Data/ASPPages/1/56.aspx")

# Base site address to use with the harvested links
root_address = 'https://www.idealchoicehomes.co.uk'

# Check that the request was successful 
print("The request was {}".format('successful' if links_page.status_code == 200 else 'unsuccessful'))

The request was successful


## Web Data Scraping: stage 2

Find all the `a` link tags in the page, then filter further by checking for a `href` attribute, and where present does it contain the prefix that all the paginated links contain.  All these links are added to a single list.

In [3]:
# Create the BS4 object from the retrieved page from the block above
soup = BeautifulSoup(links_page.content, 'html.parser')

# When looking for links, we are looking for 'a' tags that have the href_prefix below.
href_prefix = '/Data/ASPPages/1/56.aspx?CurrentPage='

# Retrieve all the 'a' elements within the page
links = soup.find_all('a')

# Filter all the 'a' elements to find those with the href attribute and the prefix within the href attribute
pages_with_links = [matching_link['href'] for matching_link in links 
                      if 'href' in matching_link.attrs and href_prefix in matching_link['href']]

# Add the first page manually, as it doesn't have an 'a' tag when it is the already selected page
pages_with_links.append('/Data/ASPPages/1/56.aspx?CurrentPage=1')

# Remove duplicates, for some reason page 2 is duplicated, unsure why, fast way to remove
pages_with_links = list(set(pages_with_links))

## Web Data Scraping: stage 3

In the block below, each of the web pages in the `pages_with_link` list is visited, and the links on each page that have the letting cycle infix `LettingCycleID` is collated into a list.

In [4]:
# For every page in the page_with_links list, we will find the lettingcycle links, as each of these pages should 
#   have letting data.

# The signature we're looking for in the href attribute of 'a' tags to indicate a lettings cycle page link
letting_infix = "LettingCycleID="

# The list that will hold the letting cycle links
letting_cycle_links = []

# Iterate through all the paginated pages, and add the multiple links per page to letting information
for current_link in pages_with_links:
    # Retrieve the page, parse, and search the 'a' tags for the letting_infix, we assume each page will resolve OK
    #  there is no validation of this.
    numbered_page = requests.get(root_address + current_link)
    soup = BeautifulSoup(numbered_page.content, 'html.parser')
    letting_cycle_links = ([matching_link['href'] for matching_link in soup.find_all('a')
                           if 'href' in matching_link.attrs and letting_infix in matching_link['href']] +
                           letting_cycle_links)


## Web Data Scraping: stage 4


In the block below we create an empty data frame with columns headings, then populate it by iterating through the letting cycle links, finding the table rows that contain data, and appending them to the data frame.  There are multiple tables on the target pages, and so we are looking for those that have `nine` `td` elements - this assumes that all pages have the same table structure, and that it hasn't changed over time.  It's crude, maybe change to look for table elements instead, and harvest data where the columns have matching headers.

In [5]:
# The column headers for the data frame
df_columns = ['area',
              'band',
              'date',
              'bids',
              'type',
              'floor',
              'bedrooms',
              'sheltered',
              'ref',
              'cycle_start',
              'cycle_end']

# The creating of the housing data frame
housing_df = pd.DataFrame(columns=df_columns)

# For every letting cycle link, follow and harvest the data
for current_letting_cycle in letting_cycle_links:
    
    # Print out the page that's currently being accessed to provide some feedback to the user about what is
    #  currently happening.
    print("On page: {}".format(current_letting_cycle))
    
    # Access the letting cycle data page given by the current letting cycle link, and create the BeautifulSoup
    #   object.
    data_page = requests.get(root_address + current_letting_cycle)
    soup = BeautifulSoup(data_page.content, 'html.parser')

    # Find the letting cycle dates by checking all paragraphs until the matching one is found, then extract dates
    #  to apply to all elements in the pages table
    paragraphs = soup.find_all('p')
    for current_paragraph in paragraphs:
        if 'The results for properties advertised' in current_paragraph.text:
            # Use the source rather than converted date, it tends to mix d/m/y m/d/y formats
            dates = list(datefinder.find_dates(current_paragraph.text, source=True))
            if len(dates) == 2:
                cycle_start = str(dates[0][1])
                cycle_end = str(dates[1][1])
    
                # Find all the table rows, 'tr' elements, and with them iterate through their 'td' elements.  Some pages 
                #  don't have tables for whatever reason, and so it is necessary to check that tr elements exist, and that
                #  td elements exist within them.
                table_rows = soup.find_all('tr')

                if len(table_rows) > 0:

                    # Iterate through all the table rows, and check that the one being looked at has nine elements as this 
                    #   matches the structure of the data tables (todo: change to find tables, and check the headers rather than
                    #   element size).
                    for current_row in table_rows:

                        row_columns = current_row.find_all('td')

                        if len(row_columns) == 9:
                            # We expect each table to have exactly the same columns as each other, and so can access them
                            #  using their position.
                            row_area = row_columns[0].string
                            row_band = row_columns[1].string
                            row_date = row_columns[2].string
                            row_bids = row_columns[3].string
                            row_type = row_columns[4].string
                            row_floor = row_columns[5].string
                            row_bedrooms = row_columns[6].string
                            row_sheltered = row_columns[7].string
                            row_ref = row_columns[8].string
                            housing_df = housing_df.append({
                                'area': row_area,
                                'band': row_band,
                                'date': row_date,
                                'bids': row_bids,
                                'type': row_type,
                                'floor': row_floor,
                                'bedrooms': row_bedrooms,
                                'sheltered': row_sheltered,
                                'ref': row_ref,
                                'cycle_start': cycle_start,
                                'cycle_end': cycle_end
                            }, ignore_index=True)


On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630191
On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630190
On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630189
On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630188
On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630187
On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630186
On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630185
On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630184
On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630183
On page: /Data/ASPPages/1/56.aspx?CurrentPage=28&LettingCycleID=630182
On page: /Data/ASPPages/1/56.aspx?CurrentPage=27&LettingCycleID=630203
On page: /Data/ASPPages/1/56.aspx?CurrentPage=27&LettingCycleID=630202
On page: /Data/ASPPages/1/56.aspx?CurrentPage=27&LettingCycleID=630201
On page: /Data/ASPPages/1/56.aspx?CurrentPage=27&LettingCycleID=630200
On pag

On page: /Data/ASPPages/1/56.aspx?CurrentPage=30&LettingCycleID=630163
On page: /Data/ASPPages/1/56.aspx?CurrentPage=30&LettingCycleID=630162
On page: /Data/ASPPages/1/56.aspx?CurrentPage=30&LettingCycleID=630161
On page: /Data/ASPPages/1/56.aspx?CurrentPage=30&LettingCycleID=630160
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630129
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630128
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630127
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630126
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630125
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630124
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630123
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630122
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630121
On page: /Data/ASPPages/1/56.aspx?CurrentPage=34&LettingCycleID=630120
On pag

On page: /Data/ASPPages/1/56.aspx?CurrentPage=10&LettingCycleID=630366
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630243
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630242
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630241
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630240
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630239
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630238
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630237
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630236
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630235
On page: /Data/ASPPages/1/56.aspx?CurrentPage=23&LettingCycleID=630234
On page: /Data/ASPPages/1/56.aspx?CurrentPage=16&LettingCycleID=630313
On page: /Data/ASPPages/1/56.aspx?CurrentPage=16&LettingCycleID=630312
On page: /Data/ASPPages/1/56.aspx?CurrentPage=16&LettingCycleID=630311
On pag

On page: /Data/ASPPages/1/56.aspx?CurrentPage=35&LettingCycleID=630114
On page: /Data/ASPPages/1/56.aspx?CurrentPage=35&LettingCycleID=630113
On page: /Data/ASPPages/1/56.aspx?CurrentPage=35&LettingCycleID=630112
On page: /Data/ASPPages/1/56.aspx?CurrentPage=35&LettingCycleID=630111
On page: /Data/ASPPages/1/56.aspx?CurrentPage=35&LettingCycleID=630110


## Web Data Scraping: stage 5

The block below is straightforward in that it does only one thing, writes the data frame to a CSV file on disk.

In [6]:
housing_df.to_csv('housing_information.csv',
                  index=False)